In [1]:
import random
answer = random.randint(0, 100)
print("Let\'s guess the numbers! (0~100)")
guess_number = input("Please input you guess!\n")
score = 100
while(True):
    if guess_number.isdigit() == False:
        guess_number = input("Please input a number! Try again!\n")
        score -=10
        continue
    else:
        guess_number = int(guess_number)
    if guess_number > answer:
        score -=5
        guess_number = input("Your guess is too big. Try again!\n")
    elif guess_number < answer:
        score -=5
        guess_number = input("Your guess is too small. Try again!\n")
    else:
        print("Congratulations! You guessed it!\n" + "You score is" , score)
        if input("Play again? [1]yes [2]no\n") == "1":
            score = 100
            guess_number = input("Please input you guess!\n")
        else:
            break   

Let's guess the numbers! (0~100)
Please input you guess!
67
Your guess is too small. Try again!
80
Your guess is too small. Try again!
94
Your guess is too big. Try again!
86
Congratulations! You guessed it!
You score is 85
Play again? [1]yes [2]no
2


In [1]:
import time

count_down = 5# 设置倒计时时间，单位：秒
interval = count_down/50 # 设置屏幕刷新的间隔时间，单位：秒
for i in range(0, int(count_down/interval)+1):
    print("\r"+"▇"*i+" "+str(i*100/count_down*interval)+"%", end="")
    time.sleep(interval)

▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 100.0%

In [2]:
import os
import json
import ibm_boto3
from ibm_botocore.client import Config, ClientError

# Constants for IBM COS values
COS_ENDPOINT = 'https://s3.hkg02.cloud-object-storage.appdomain.cloud' 
COS_API_KEY_ID = ""
COS_AUTH_ENDPOINT = "https://iam.cloud.ibm.com/identity/token"
COS_RESOURCE_CRN = ""

cos = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint=COS_AUTH_ENDPOINT,
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

def get_buckets():
    print("Retrieving list of buckets")
    try:
        buckets = cos.buckets.all()
        for bucket in buckets:
            return "Bucket Name: {0}".format(bucket.name)
    except ClientError as be:
        return "CLIENT ERROR: {0}\n".format(be), 'Error'
    except Exception as e:
        return "Unable to retrieve list buckets: {0}".format(e), 'Error'

def get_item(bucket_name,item_name):
    print("Retrieving item from bucket: {0}, key: {1}".format(bucket_name, item_name))
    try:
        file = cos.Object(bucket_name, item_name).get()
        return file["Body"].read()
    except ClientError as be:
        return "CLIENT ERROR: {0}\n".format(be), 'Error'
    except Exception as e:
        return "Unable to retrieve file contents: {0}".format(e), 'Error'
        
def get_bucket_contents(bucket_name):
    print("Retrieving bucket contents from: {0}".format(bucket_name))
    try:
        files = cos.Bucket(bucket_name).objects.all()
        dict={}
        for file in files:
        	dict[file.key]="{} bytes".format(file.size)
        return dict
    except ClientError as be:
        return "CLIENT ERROR: {0}\n".format(be), 'Error'
    except Exception as e:
        return "Unable to retrieve bucket contents: {0}".format(e), 'Error'

def multi_part_upload(bucket_name, item_name, file_path):
    try:
        print("Starting file transfer for {0} to bucket: {1}\n".format(item_name, bucket_name))
        # set 2.5 MB chunks
        part_size = 1024 * 1024 * 2.5

        # set threadhold to 10 MB
        file_threshold = 1024 * 1024 * 10

        # set the transfer threshold and chunk size
        transfer_config = ibm_boto3.s3.transfer.TransferConfig(
            multipart_threshold=file_threshold,
            multipart_chunksize=part_size
        )

        # the upload_fileobj method will automatically execute a multi-part upload
        # in 2.5 MB chunks for all files over 10 MB
        with open(file_path, "rb") as file_data:
            cos.Object(bucket_name, item_name).upload_fileobj(
                Fileobj=file_data,
                Config=transfer_config
            )

        print("Transfer for {0} Complete!\n".format(item_name))
    except ClientError as be:
        print("CLIENT ERROR: {0}\n".format(be))
    except Exception as e:
        print("Unable to complete multi-part upload: {0}".format(e))

def download(bucket_name,item_name):
	file = get_item(bucket_name,item_name)
	if file[1] != 'Error':
		with open(os.getcwd()+'//'+item_name, 'wb+') as f:
			f.write(file)
		date = json.loads(file)
		return date

if __name__ == '__main__':
	download('pythonuiu','cos-python-sdk-packages.tar.gz')
	multi_part_upload('pythonuiu')
	print(get_bucket_contents('pythonuiu'))

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 93)